In [1]:
import sys
import argparse

import torch
import torch.nn as nn
from torchtext.legacy import data

In [2]:
class RNNClassifier(nn.Module):
    def __init__(self,
                 input_size,
                 word_vec_size,
                 hidden_size,
                 n_classes,
                 n_layers = 4,
                 dropout_p = .3
                ):
        self.vocab_size = input_size # vocabulary_size
        self.word_vec_size = word_vec_size
        self.hidden_size = hidden_size
        self.n_classes = n_classes
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        
        super().__init__()
        
        self.emb = nn.Embedding(input_size, word_vec_size)
        self.rnn = nn.LSTM(input_size = word_vec_size,
                           hidden_size = hidden_size,
                           num_layers = n_layers,
                           dropout = dropout_p,
                           batch_first = True,
                           bidirectional = True
                          )
        self.generator = nn.Linear(hidden_size * 2, n_classes)
        # We use LogSoftmax + NLLLoss instead of Softmax + CrossEntropy
        self.activation = nn.LogSoftmax(dim = -1)
        
    def forward(self, x):
        # /x/ = (batch_size, length)
        x = self.emb(x)
        # /x/ = (batch_size, length, word_vec_dim)
        x, _ = self.rnn(x)
        # /x/ = (batch_size, length, hidden_size * 2)
        y = self.activation(self.generator(x[:, -1]))
        # /y/ = (batch_size, n_classes)
            
        return y
    
class Config():
    def __init__(self):
        self.model_fn = "./models/model.pth"
        self.train_fn = "./data/processed_data.txt"
        
        self.gpu_id = -1
        self.verbose = 2
        
        self.min_vocab_freq = 5
        self.max_vocab_size = 999999
        
        self.batch_size = 256
        self.n_epochs = 10
        
        self.word_vec_size = 256
        self.dropout = .3
        
        self.max_length = 256
        
        self.rnn = True
        self.hidden_size = 512
        self.n_layers = 4
        
        self.cnn = False
        self.use_batch_norm = True
        self.window_sizes = [3, 4, 5]
        self.n_filters = [100, 100, 100]

In [3]:
def read_text(input_string, max_length=256):
    '''
    Read text from standard input for inference.
    '''
    lines = []

    for line in input_string: # sys.stdin:
        if line.strip() != '':
            lines += [line.strip().split(' ')[:max_length]]

    return lines

def define_field():
    '''
    To avoid use DataLoader class, just declare dummy fields. 
    With those fields, we can retore mapping table between words and indice.
    '''
    return (
        data.Field(
            use_vocab=True,
            batch_first=True,
            include_lengths=False,
        ),
        data.Field(
            sequential=False,
            use_vocab=True,
            unk_token=None,
        )
    )

In [4]:
class test_Config():
    def __init__(self):
        self.model_fn = "./models/model.pth"
        self.gpu_id = -1
        self.batch_size = 256
        self.top_k = 1
        self.max_length = 256
        
        self.drop_rnn = False
        self.drop_cnn = True

test_config = test_Config()

In [35]:
def Load_Model(input_string, test_rate):
    
    # test
    test_value = 0
    
    saved_data = torch.load(
        test_config.model_fn,
        map_location='cpu' if test_config.gpu_id < 0 else 'cuda:%d' % test_config.gpu_id
    )
    
    train_config = saved_data['config']
    rnn_best = saved_data['rnn']
    cnn_best = saved_data['cnn']
    vocab = saved_data['vocab']
    classes = saved_data['classes']

    vocab_size = len(vocab)
    n_classes = len(classes)

    text_field, label_field = define_field()
    text_field.vocab = vocab
    label_field.vocab = classes

    lines = read_text(input_string, max_length=test_config.max_length)

    with torch.no_grad():
        ensemble = []
        if rnn_best is not None and not test_config.drop_rnn:
            # Declare model and load pre-trained weights.
            model = RNNClassifier(
                input_size=vocab_size,
                word_vec_size=train_config.word_vec_size,
                hidden_size=train_config.hidden_size,
                n_classes=n_classes,
                n_layers=train_config.n_layers,
                dropout_p=train_config.dropout,
            )
            model.load_state_dict(rnn_best)
            ensemble += [model]
        if cnn_best is not None and not test_config.drop_cnn:
            # Declare model and load pre-trained weights.
            model = CNNClassifier(
                input_size=vocab_size,
                word_vec_size=train_config.word_vec_size,
                n_classes=n_classes,
                use_batch_norm=train_config.use_batch_norm,
                dropout_p=train_config.dropout,
                window_sizes=train_config.window_sizes,
                n_filters=train_config.n_filters,
            )
            model.load_state_dict(cnn_best)
            ensemble += [model]

        y_hats = []
        # Get prediction with iteration on ensemble.
        for model in ensemble:
            if test_config.gpu_id >= 0:
                model.cuda(test_config.gpu_id)
            # Don't forget turn-on evaluation mode.
            model.eval()

            y_hat = []
            for idx in range(0, len(lines), test_config.batch_size):                
                # Converts string to list of index.
                x = text_field.numericalize(
                    text_field.pad(lines[idx:idx + test_config.batch_size]),
                    device='cuda:%d' % test_config.gpu_id if test_config.gpu_id >= 0 else 'cpu',
                )

                y_hat += [model(x).cpu()]
            # Concatenate the mini-batch wise result
            y_hat = torch.cat(y_hat, dim=0)
            # |y_hat| = (len(lines), n_classes)

            y_hats += [y_hat]

            model.cpu()
        # Merge to one tensor for ensemble result and make probability from log-prob.
        y_hats = torch.stack(y_hats).exp()
        # |y_hats| = (len(ensemble), len(lines), n_classes)
        y_hats = y_hats.sum(dim=0) / len(ensemble) # Get average
        # |y_hats| = (len(lines), n_classes)

        probs, indice = y_hats.topk(test_config.top_k)

        for i in range(len(lines)):
            sys.stdout.write('[%s, 점수:%s] %s\n' % (
                ' '.join(['긍정' if int(classes.itos[indice[i][j]]) > 3 else '부정' for j in range(test_config.top_k)]), 
                ' '.join([classes.itos[indice[i][j]] for j in range(test_config.top_k)]), 
                ' '.join(lines[i])
            ))
            # test
            if ((int(classes.itos[indice[i][0]]) > 3) and test_rate==1) or ((int(classes.itos[indice[i][0]]) < 3) and test_rate==0):
                test_value += 1
    return test_value

In [6]:
import pandas as pd

data_file = pd.read_table('./data/ratings_test.txt') # DataFrame 으로 불러오기
print(data_file)
print('총 샘플의 수 : ', len(data_file))
print(data_file[:100])

            id                                           document  label
0      6270596                                                굳 ㅋ      1
1      9274899                               GDNTOPCLASSINTHECLUB      0
2      8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0
3      6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
4      6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0
...        ...                                                ...    ...
49995  4608761          오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함      1
49996  5308387       의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO      0
49997  9072549                 그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다      0
49998  5802125     절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네      0
49999  6070594                                         마무리는 또 왜이래      0

[50000 rows x 3 columns]
총 샘플의 수 :  50000
         id                                           document  label
0   6270596

In [28]:
# data_file = data_file['document']
# print(data_file)

test_value = 0

In [8]:
data_file.drop_duplicates(subset=['document'], inplace=True) # 중복제거
data_file['document'] = data_file['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True) # 기타 문자 제거
data_file = data_file.dropna(how = 'any') # null값 제거
print(data_file[:100])
print('총 샘플의 수 : ', len(data_file))
input_string = data_file['document'].values.tolist()
print(input_string[:10])

         id                                           document  label
0   6270596                                                굳 ㅋ      1
1   9274899                                                         0
2   8544678                   뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아      0
3   6825595                          지루하지는 않은데 완전 막장임 돈주고 보기에는      0
4   6723715          만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠      0
..      ...                                                ...    ...
95  6753658                                 전기톱은못들고다니는데 엔진톱이겠죠      0
96  9665771                                     완전 재밌엇는데 왜 평점이      1
97  8757576                                     제임스 완이 내 목표임 ㄷ      1
98  9850643  점고 아깝다 개막장 영화의 원조라고나 할까아내와 사별한 지 얼마나 지났다고 딴 여자...      0
99  9985753  가끔 문득 생각나서 다시보는 영화색감이 정말 예술이죠 강렬하고 화려하고츠지야안나의 ...      1

[100 rows x 3 columns]
총 샘플의 수 :  49157
['굳 ㅋ', '', '뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아', '지루하지는 않은데 완전 막장임 돈주고 보기에는', '만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠

In [41]:
# input_string = [
#     "이건 쓰레기야",
#     "최고에요! ㅎㅎ",
#     "구매하길 잘 했네요",
#     "실망이에요.. 금방 망가지네요",
#     "다시는 구매 안합니다.",
#     "또 주문할게요.",
#     "포장도 제대로 안되어있고 정말 엉망이네요",
#     "내돈 돌려줘",
#     "사이즈가 안맞아요",
#     "따듯하고 좋아요 세탁하면 어떨지 아직은 모르겠네요 세탁후 줄어듬만 없으면 100점이에요",
#     "이정도면 무난히 만족합니다. 다른분들은 모르겠지만 딱 제가 찾던 폴라티네요.",
#     "남들이 욕하길래 정말 궁금해서 사봤어요 우와 세상에ㅎㅎ 잠옷이 생겼네요 요즘 좀 우울했는에 웃음이 나오네요ㅋㅋ 옷감은 무지 얇고 목은 왜이리 긴지 난 목이기니까 하면서 샀는데 어느나라에 목이 긴 부족에게 추천하고 싶네요 돈많고 좀 우울한 분에게 기분전환용으로 추천합니다 덕분에 웃었어요",
#     "다신 안삼",
#     "인연끊고 싶은 사람에게 추천해라",
#     "이런 입지도 못하는 옷을 팔다니 싸더라도 입을 수 있게는 만들어야지 기장이 짧아서 배꼽이 보일지경ㅜㅜ",
#     "한마디로 잘라 말하면 사지 마세요.",
#     "너무셔요 너무셔서애기가안먹네요 다른제품은잘먹었는데.. 제구매는없을듯요",
#     "오배송 5단계주문했는데 4단계왔습니다 안보고 모르고 한팩 뜯어썻네요 그거 제외하고 다시 교환해주세요",
#     "너무느려요 배송너무느려서 주문못하겠네요",
#     "정말 박스 너덜너덜하고 기저귀 터져서왔네요 제가 어지간해선 리뷰 귀찮아서 안남기는데 제개 보낸 기저귀는 박스도 재포장해서 너덜너덜해져서 거의 분리된상태로 온데다가 안에 뜯어보니 박스가 그상태로 굴러와서인지 원래 뜯어진건지, 시킨것중 기저귀한팩 바닥이 반쯤 뜯어져서 왔네요. 박스도 첨부터 너덜너덜해서 급히 그부분 보수해서 테이핑해서 온데다가, 안에 내용물까지 저모냥인데 그것도 다른게아니라 아기 소중한데 닿을 기저귀인데 정말 해도해도 너무하시네요. 동영상 가지고있구요, 저도 놀고먹는사람아니라서 바빠 반품하기도 싫습니다. 다음부터는 다른건 몰라도 아기용품은 양심껏 파세요 ㅡㅡ",
#     "배송진짜느림 배송진짜느려요 급한분들은 주문하시면 안될거같아요 피드백도 느려서 재고도없이 판매하면 어쩌라는건지;;",
#     "최악 포장도 엉망 cs도엉망 배송최악",
#     "이거 정말 좋네요",
#     "배송도 빠르고 제품상태가 좋아요",
#     "민경환 바보"
# ]

# print(input_string)

# input_string = [
#     ""
# ]
test_value = 0
test_count = 200
print (str(test_value))
for i in range(test_count):
    if (input_string[i] != ""):
        print('긍정' if data_file['label'][i] == 1 else '부정', end=' ')
        ii = []
        ii.append(input_string[i])
        test_value += Load_Model(ii, data_file['label'][i])
        
print (str(test_value/test_count * 100) + "%")

0
긍정 [긍정, 점수:5] 굳 ㅋ
부정 [부정, 점수:2] 뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아
부정 [부정, 점수:2] 지루하지는 않은데 완전 막장임 돈주고 보기에는
부정 [부정, 점수:2] 만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠
긍정 [긍정, 점수:5] 음악이 주가 된 최고의 음악영화
부정 [부정, 점수:2] 진정한 쓰레기
부정 [부정, 점수:2] 마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가고개를 젖게한다
부정 [부정, 점수:1] 갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한남무 아 그립다 동사서독같은 영화가 이건 류아류작이다
긍정 [부정, 점수:2] 이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨  모든 사람이 그렇지는 않네
긍정 [긍정, 점수:5] 괜찮네요오랜만포켓몬스터잼밌어요
부정 [부정, 점수:2] 한국독립영화의 한계 그렇게 아버지가 된다와 비교됨
긍정 [부정, 점수:2] 청춘은 아름답다 그 아름다움은 이성을 흔들어 놓는다 찰나의 아름다움을 잘 포착한 섬세하고 아름다운 수채화같은 퀴어영화이다
긍정 [부정, 점수:2] 눈에 보이는 반전이었지만 영화의 흡인력은 사라지지 않았다
부정 [부정, 점수:1] 스토리 연출 연기 비주얼 등 영화의 기본 조차 안된 영화에 무슨 평을 해 이런 영화 찍고도 김문옥 감독은 내가 영화 경력이 몇인데 조무래기들이 내 영화를 평론해 같은 마인드에 빠져있겠지
긍정 [부정, 점수:2] 소위 ㅈ문가라는 평점은 뭐냐
긍정 [긍정, 점수:5] 최고
부정 [부정, 점수:1] 발연기 도저히 못보겠다 진짜 이렇게 연기를 못할거라곤 상상도 못했네
긍정 [긍정, 점수:5] 나이스
부정 [부정, 점수:2] 별 재미도없는거 우려먹어  챔프에서 방송 몇번했더라  ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
긍정 [긍정, 점수:5] 일의 금요일 나이트메어시리즈와 함께 가장 많은 시리즈를 양산해냈던 헬레이저 시리즈의 첫편 작가의 상상력이 돋보이는 작품이며 갈고리로 사지찢는 